构造数据表，写入大量数据

In [ ]:
# 建表
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
engine = create_engine("sqlite:///test3.db", echo=True)

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    description: Mapped[str] = mapped_column(String(240))

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


Base.metadata.create_all(engine)


In [ ]:
import random
import string   
def gen_name():
    length = random.randint(3, 9)
    name = ''.join(random.choices(string.ascii_lowercase, k=length))
    return name.capitalize()


from sqlalchemy.orm import Session
with Session(engine) as session:
    for _ in range(1000):
        name = gen_name()
        new_user = User(
            name = name,
            fullname = name+' '+gen_name(),
            description="for test..."
            )
        session.add(new_user)
    session.commit()

In [ ]:
with Session(engine) as s:
    users = s.query(User).all()

for user in users:
    print(user)

In [ ]:
%%timeit
with Session(engine) as s:
    u = s.query(User).filter_by(name='Lknom').first()
print(u)

In [ ]:
%%timeit

from sqlalchemy import select
stmt = select(User).where(User.name == 'Lknom')

with Session(engine) as s:
    u = session.scalars(stmt).one()
print(u)

In [ ]:
%%timeit
from sqlalchemy import text
query = text("SELECT * FROM user_account WHERE name = :name")

with Session(engine) as s:
    result = s.execute(query, {'name': "Lknom"})

# 获取查询结果
for row in result:
    print(row)